In [7]:
import tlsh
import random
import numpy
from scipy.spatial import distance

# Get the histogram of buckets

In [3]:
#t1 = tlsh.hash(random.randbytes(1024))
t1 = 'T1EB36D022C140901AE8F756FD42BA43BAFCB46E31357498F77DEBE918C42B694C9106B7'
print(len(t1),t1)
ex = t1[8:]
print(len(ex),ex)

72 T1EB36D022C140901AE8F756FD42BA43BAFCB46E31357498F77DEBE918C42B694C9106B7
64 22C140901AE8F756FD42BA43BAFCB46E31357498F77DEBE918C42B694C9106B7


In [4]:
def to_bucket(hex):
    return int(bin(int(hex,16))[2:4],2), int(bin(int(hex,16))[4:6],2)

#[to_bucket(hex) for hex in ex]
# print(to_bucket(ex[0]))
print(bin(int(ex[0],16)))
print(bin(int(ex[0],16))[4:6])

0b10



In [5]:
def to_hist(t):
    buckets = numpy.zeros(128, dtype=numpy.int32)
    for idx, (sixteen, one) in enumerate(zip(t[0::2], t[1::2])):
        buckets[4 * idx] = (int(sixteen,16) & 0x0C) >> 2
        buckets[4 * idx + 1] = (int(sixteen,16) & 0x03)
        buckets[4 * idx + 2] = (int(one,16) & 0x0C) >> 2
        buckets[4 * idx + 3] = (int(one,16) & 0x03)
    return buckets

In [8]:
buk = to_hist(ex)
print(len(buk))
buk

128


array([0, 2, 0, 2, 3, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 2, 3, 2,
       2, 0, 3, 3, 1, 3, 1, 1, 1, 2, 3, 3, 3, 1, 1, 0, 0, 2, 2, 3, 2, 2,
       1, 0, 0, 3, 2, 3, 2, 2, 3, 3, 3, 0, 2, 3, 1, 0, 1, 2, 3, 2, 0, 3,
       0, 1, 0, 3, 1, 1, 1, 3, 1, 0, 2, 1, 2, 0, 3, 3, 1, 3, 1, 3, 3, 1,
       3, 2, 2, 3, 3, 2, 2, 1, 0, 1, 2, 0, 3, 0, 1, 0, 0, 2, 2, 3, 1, 2,
       2, 1, 1, 0, 3, 0, 2, 1, 0, 1, 0, 0, 1, 2, 2, 3, 1, 3], dtype=int32)

# A Hamming approximation of TLSH distance

In [56]:
def hamming_three(buckets):
    ham = numpy.zeros(3 * 128, dtype=numpy.bool_)
    for idx, val in enumerate(buckets):
        if val == 3:
            ham[3 * idx + 2] = 1
        if val >= 2:
            ham[3 * idx + 1] = 1
        if val >= 1:
            ham[3 * idx] = 1
    return ham

In [115]:
# hamming_three(buk)

In [59]:
t2 = tlsh.hash(random.randbytes(1024))
ex2 = t2[8:]

In [71]:
buk2 = to_hist(ex2)

In [72]:
distance.hamming(hamming_three(buk),hamming_three(buk2)) * 3 * 128

155.0

# Another Hamming Distance, weighting extremes more

In [69]:
def hamming_five(buckets):
    ham = numpy.zeros(5 * 128, dtype=numpy.bool_)
    for idx, val in enumerate(buckets):
        if val == 3:
            ham[5 * idx + 4] = 1
            ham[5 * idx + 3] = 1
        if val >= 2:
            ham[5 * idx + 2] = 1
        if val < 1:
            ham[5 * idx + 1] = 1
            ham[5 * idx] = 1
    return ham

In [102]:
distance.hamming(hamming_five(buk),hamming_five(buk2)) * 5 * 128

253.0

# Another Hamming Distance (six), weighting extremes more

In [109]:
def hamming_six(buckets):
    ham = numpy.zeros(6 * 128, dtype=numpy.bool_)
    for idx, val in enumerate(buckets):
        if val == 3:
            ham[6 * idx + 5] = 1
            ham[6 * idx + 4] = 1
            ham[6 * idx + 3] = 1
            ham[6 * idx + 2] = 1
        if val >= 2:
            ham[6 * idx + 1] = 1
        if val > 1:
            ham[6 * idx] = 1
    return ham

distance.hamming(hamming_six(buk),hamming_six(buk2)) * 6 * 128

290.0

# TLSH

In [114]:
tlsh.diff(t1,t2)

217

In [113]:
import tlsh_distance
distance_bodies(t1[2:],t2[2:])

186

In [107]:
distance_headers(t1[2:],t2[2:])

25

In [98]:
get_bin = lambda x, n: format(x, 'b').zfill(n)
a = get_bin(2,4)
b = get_bin(15,4) 
print(a,b)

#tlsh distance on body	
d1 = 6 if (abs(int(a,2)//4	- int(b,2)//4) == 3) else  abs(int(a,2)//4	- int(b,2)//4) 
d2 = 6 if (abs(int(a,2)%4	- int(b,2)%4) ==3) else abs(int(a,2)%4	- int(b,2)%4)      
print("tlsh_distance on body:", d1+d2, (d1,d2))          #8

0010 1111
tlsh_distance on body: 7 (6, 1)


In [101]:
def hamming_distance1(chaine1, chaine2):
	return sum(c1 != c2 for c1, c2 in zip(chaine1, chaine2))
	
def hamming_distance2(chaine1, chaine2):
	return len(list(filter(lambda x : ord(x[0])^ord(x[1]), zip(chaine1, chaine2))))

#hamming distance for body	
print("hamming_distance1:", hamming_distance1(a,b))  
print("hamming_distance2:", hamming_distance2(a,b))  

#eculidance distance for body	
print("l2 distance :", abs(int(a,2) - int(b,2)))    
print("adjust_l2_distance :", abs(int(a,2)//4 - int(b,2)//4) + abs(int(a,2)%4 - int(b,2)%4))    

hamming_distance1: 3
hamming_distance2: 3
l2 distance : 13
adjust_l2_distance : 4
